### 최적의 알고리즘 찾기

    - all_estimators() 메소드 이용하여 모든 알고리즘 추출
    
    [참고도서] 파이썬을 이용한 머신러닝, 딥러닝 실전 앱 개발 (위키북스)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import all_estimators

import warnings
warnings.filterwarnings('ignore')

# 붓꽃 데이터 읽어 들이기
iris_data = pd.read_csv("../data/iris/iris.csv", encoding="utf-8")

# 붓꽃 데이터를 레이블과 입력 데이터로 분리하기 
y = iris_data.loc[:,"variety"]
x = iris_data.loc[:,["sepal.length","sepal.width","petal.length","petal.width"]]


In [9]:


# (0) 학습 전용과 테스트 전용 분리하기 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, train_size = 0.8, shuffle = True)

# (1) classifier 알고리즘 모두 추출하기
allAlgorithms = all_estimators(type_filter="classifier")

for(name, algorithm) in allAlgorithms:
    # (1) 알고리즘 종류 확인
    # print(name)
    
    # 현재 자료형이 안 맞아서 학습되지 않는 알고리즘들이기에 제외해야 한다 -> 추가로 코딩을 예쁘게 
    if name=='CheckingClassifier' or name=='ClassifierChain' \
        or name=='MultiOutputClassifier' or name=='OneVsOneClassifier' \
        or name=='OneVsRestClassifier' or name=='OutputCodeClassifier' \
        or name=='FixedThresholdClassifier' or name=='TunedThresholdClassifierCV'\
        or name=='VotingClassifier' or name=='StackingClassifier':
        continue
        
    #-----------------------------------------    
    # 에러가 발생하면 위에 print(name)으로 알고리즘을 확인 후 에러난 알고리즘이름을 위에 추가한다
    
    # (2) 각 알고리즘 객체 생성하기 
    clf = algorithm()

    # (3) 학습하고 평가하기 
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(name,"의 정답률 = " , accuracy_score(y_test, y_pred))

AdaBoostClassifier 의 정답률 =  0.9333333333333333
BaggingClassifier 의 정답률 =  0.9666666666666667
BernoulliNB 의 정답률 =  0.23333333333333334
CalibratedClassifierCV 의 정답률 =  0.9333333333333333
CategoricalNB 의 정답률 =  0.9
ComplementNB 의 정답률 =  0.6666666666666666
DecisionTreeClassifier 의 정답률 =  0.9666666666666667
DummyClassifier 의 정답률 =  0.23333333333333334
ExtraTreeClassifier 의 정답률 =  0.9333333333333333
ExtraTreesClassifier 의 정답률 =  0.9333333333333333
GaussianNB 의 정답률 =  0.9666666666666667
GaussianProcessClassifier 의 정답률 =  0.9333333333333333
GradientBoostingClassifier 의 정답률 =  0.9333333333333333
HistGradientBoostingClassifier 의 정답률 =  0.9333333333333333
KNeighborsClassifier 의 정답률 =  1.0
LabelPropagation 의 정답률 =  0.9333333333333333
LabelSpreading 의 정답률 =  0.9333333333333333
LinearDiscriminantAnalysis 의 정답률 =  1.0
LinearSVC 의 정답률 =  0.9333333333333333
LogisticRegression 의 정답률 =  0.9333333333333333
LogisticRegressionCV 의 정답률 =  1.0
MLPClassifier 의 정답률 =  0.9666666666666667
MultinomialNB 의 정답률 =  0

[ 문제점 ]

각 알고리즘의 정답률을 비교했지만, 평가 횟수가 1회 밖에고 사용한 데이타도 한 가지 패턴이다.

여러 데이타 패턴으로 평가하는 방법이 교차 검증(corss-validation)이다


### K-교차 검증 (cross-validataion)

- K개의 그룹으로 분할
- K-1개의 학습 데이타
- 남은 1개의 평가 데이타
- K번 반복


In [23]:
import pandas as pd
from sklearn.utils import all_estimators
from sklearn.model_selection import KFold
import warnings
from sklearn.model_selection import cross_val_score

# 붓꽃 데이터 읽어 들이기
iris_data = pd.read_csv("../data/iris/iris.csv", encoding="utf-8")

# 붓꽃 데이터를 레이블과 입력 데이터로 분리하기 
y = iris_data.loc[:,"variety"]
x = iris_data.loc[:,["sepal.length","sepal.width","petal.length","petal.width"]]

In [25]:
# classifier 알고리즘 모두 추출하기
warnings.filterwarnings('ignore')
allAlgorithms = all_estimators(type_filter="classifier")

# (1)
# K-분할 크로스 밸리데이션 전용 객체 
# 데이타를 5그룹으로 분할하고 분할할 때 랜덤하게 섞는다
kfold_cv = KFold(n_splits=5, shuffle=True)

for(name, algorithm) in allAlgorithms:
    
    # 현재 자료형이 안 맞아서 학습되지 않는 알고리즘들이기에 제외해야 한다 -> 추가로 코딩을 예쁘게 
    if name=='CheckingClassifier' or name=='ClassifierChain' \
        or name=='MultiOutputClassifier' or name=='OneVsOneClassifier' \
        or name=='OneVsRestClassifier' or name=='OutputCodeClassifier' \
        or name=='FixedThresholdClassifier' or name=='TunedThresholdClassifierCV'\
        or name=='VotingClassifier' or name=='StackingClassifier':
        continue
        
    # (2) 각 알고리즘 객체 생성하기
    clf = algorithm()

    # (3)
    # score 메서드를 가진 클래스를 대상으로 하기
    if hasattr(clf,"score"):

        # (4) 크로스 밸리데이션
        # cross_val_score (clf:알고리즘 객체, x:입력데이터, y:레이블데이터, cv:교차검증 적용 객체)
        scores = cross_val_score(clf, x, y, cv=kfold_cv)
        print(name,"의 정답률=")
        print(scores)

AdaBoostClassifier 의 정답률=
[0.9        0.96666667 0.93333333 1.         0.93333333]
BaggingClassifier 의 정답률=
[0.96666667 0.93333333 0.96666667 0.93333333 0.93333333]
BernoulliNB 의 정답률=
[0.16666667 0.16666667 0.3        0.26666667 0.26666667]
CalibratedClassifierCV 의 정답률=
[0.96666667 0.83333333 0.96666667 0.86666667 0.93333333]
CategoricalNB 의 정답률=
[0.96666667 0.83333333 0.93333333 1.         0.9       ]
ComplementNB 의 정답률=
[0.7        0.56666667 0.8        0.53333333 0.73333333]
DecisionTreeClassifier 의 정답률=
[1.         0.9        0.9        0.96666667 0.86666667]
DummyClassifier 의 정답률=
[0.3        0.23333333 0.2        0.2        0.3       ]
ExtraTreeClassifier 의 정답률=
[0.9        0.9        0.96666667 1.         0.96666667]
ExtraTreesClassifier 의 정답률=
[0.93333333 0.96666667 0.93333333 1.         0.96666667]
GaussianNB 의 정답률=
[1.         0.93333333 0.96666667 0.93333333 0.93333333]
GaussianProcessClassifier 의 정답률=
[0.96666667 0.86666667 0.96666667 0.96666667 0.96666667]
GradientBoosting

### 최적의 매개변수 찾기

위에서는 각각의 알고리즘 객체에 디폴트 매개변수만 사용하여 결과를 확인

그러나 매개변수를 변경하면 결과가 달라지게

최적의 매개변수를 찾는 것도 중요하다

In [19]:
import pandas as pd
from sklearn.utils import all_estimators
from sklearn.model_selection import KFold
import warnings
from sklearn.model_selection import cross_val_score

# 붓꽃 데이터 읽어 들이기
iris_data = pd.read_csv("../data/iris/iris.csv", encoding="utf-8")

# 붓꽃 데이터를 레이블과 입력 데이터로 분리하기 
y = iris_data.loc[:,"variety"]
x = iris_data.loc[:,["sepal.length","sepal.width","petal.length","petal.width"]]

In [21]:
# 학습 전용과 테스트 전용 분리하기 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, train_size = 0.8, shuffle = True)

# 그리드 서치에서 사용할 매개 변수 --- (*1)
parameters = [
    {"C": [1, 10, 100, 1000], "kernel":["linear"]},
    {"C": [1, 10, 100, 1000], "kernel":["rbf"], "gamma":[0.001, 0.0001]},
    {"C": [1, 10, 100, 1000], "kernel":["sigmoid"], "gamma": [0.001, 0.0001]}
]

# 그리드 서치 --- (*2)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


kfold_cv = KFold(n_splits=5, shuffle=True)
clf = GridSearchCV( SVC(), parameters, cv=kfold_cv)
clf.fit(x_train, y_train)
print("최적의 매개 변수 = ", clf.best_estimator_)

# 최적의 매개 변수로 평가하기 --- (*3)
y_pred = clf.predict(x_test)
print("최종 정답률 = " , accuracy_score(y_test, y_pred))

최적의 매개 변수 =  SVC(C=1, kernel='linear')
최종 정답률 =  1.0
